Gruppe D

# Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.<br><br> 
Nachdem wir uns einen Überblick über die möglichen Datenquellen verschafft haben, sind wir zum Entschluss gekommen, uns die Zensus Daten von 2011 genauer anzuschauen. Die Zensus Daten 2011 enthalten unter anderem Angaben über die Erwerbstätigkeit in Deutschland sowie unterschiedlichste soziodemografische Informationen (siehe [Zensus 2011](Zensus%202011%20-%20Methoden%20und%20Verfahren.pdf)). <br><br>
Hinsichtlich der Arbeitslosigkeit gibt es diverse Vorurteile und Vermutungen. So wird häufig behauptet, dass der Grossteil der Arbeitslosen Personen mit Migrationshintergrund sind oder ein schlechtes Bildungsniveau vorweisen.<br><br>  
Die Bundeszentrale für politische Bildung veröffentlichte im Jahr 2022 hierzu einen Bericht, welcher diese zwei Vermutungen sogar bestätigt (vgl. Arbeitslosenquoten nach Geschlecht und Staatsangehoerigkeit, bpb, 2021), (vgl. Arbeitslosenquoten nach Bildung und Alter, bpb, 2021).
Diese Informationen waren für uns Anreiz genug, um diese Zusammenhänge zu untersuchen.<br><br>  
Die Fragestellung, welche wir innerhalb dieses Projekts untersuchen, lautet wie folgt:<br><br>

::: {.callout-note}
### Fragestellung
Gibt es einen Zusammenhang zwischen den soziodemografischen Merkmalen und der Arbeitslosenrate?
:::
<br>


Die Arbeitslosigkeit im Allgemeinen ist ein Indikator für die Situation auf dem Arbeitsmarkt. Die Arbeitslosenquote kann unter Angabe der "Anzahl Erwerbslosen" sowie der "Anzahl Erwerbstätigen" berechnet werden. Als soziodemografische Merkmale haben wir uns für folgende sechs entschieden und wie folgt definiert (siehe @sec-pvar). Der Grund wieso wir uns für die 6 Merkmale entschieden haben war, da aufgrund der Zensus Umfrage die soziodemografischen Merkmale in verschiedene Kategorien aufgeteilt waren. Dies sind Informationen auf Gemeindeebene, wie z.B. Angaben zum Familienstand ((Anzahl Personen die ledig sind, verheiratet, usw.), Angaben zur Altersstruktur ((Anzahl Personen die unter 10 Jahre alt sind,zwischen 10-19 Jahre alt sind, usw.), Angaben nach Relegionszugehörigkeit ((Anzahl Personen die römisch-katholisch, evangelisch oder sonstiges) oder Angaben zum Bildungsniveau (Anzahl Personen ohne beruflischen Abschluss, mindestens eine Lehre, mindestens Hochschulabschluss, usw.). Aufgrund dieser Kategorien entschieden wir uns Quoten zu bilden, die unserer Meinung nach in einer Beziehung zur Arbeitslosenquote stehen können. <br><br>  

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

Zur Visualisierung der Beziehungen zwischen Response- und Predictor Variables haben wir uns für die Anwendung von Streudiagrammen bzw. Scatter Plots entschieden. Jeden Prädikator haben wir mit der Arbeitslosenquote2 (bereinigte Arbeitslosenquote) gegegnübergestellt, um erste Erkenntnisse gewinnen zu können (siehe @sec-EDArel). Jedoch ist es anhand der Scatter Plots zunächst schwierig zu erkennen, welcher Prädikator die höchste Korrelation mit der Arbeitslosigkeit aufweist, da die berechneten Variablen unterschiedliche Werte bzw. Verteilungen auf der X-Achse annehmen und kein direkter Vergleich stattfinden kann.  <br><br> 
Zu erkennen ist, dass lediglich die Variable "Christenquote" eine moderate (über 0,5 bzw. unter -0,5) Korrelation mit der Arbeitslosenquote aufweist. Zwei weitere Quoten, die noch eine leichte bis moderarte Korrelation haben sind die "Singlequote" sowie die "Migrationsquote".<br><br>   




## Import relevanter Module

In [1]:
#| echo: true 
#| code-fold: true
#import relevant modulues
import pandas as pd
import altair as alt
import numpy as np
from pandas import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows() #aus Code overview Histogramm
from scipy import stats # to compute the mode 

from sklearn.linear_model import LinearRegression #Fitting a line
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SequentialFeatureSelector

import matplotlib.pyplot as plt  # To visualize

import joblib
import time

In [2]:
def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

## Import Datensatz

In [3]:
#| echo: true
#| code-fold: true
df_bevoelkerung = pd.read_csv(
    '../references/csv_Bevoelkerung/Zensus11_Datensatz_Bevoelkerung.csv',
    delimiter=';',
    dtype={
        'AGS_12': 'category',
        'RS_Land': 'category',
        'RS_RB_NUTS2': 'category',
        'RS_Kreis': 'category',
        'RS_VB': 'category',
        'RS_Gem': 'category',
        'Name': 'category',
        'Reg_Hier': 'category'
    },
    low_memory= False #um Warnung zu verhindern
)

## Data Structure

Um einen ersten groben Überblick zu bekommen, geben wir uns eine Info über unseren Datensatz mit der Pandas-Funktion `pd.info()` aus.

In [4]:
#| echo: true
#| code-fold: true
df_bevoelkerung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Columns: 223 entries, AGS_12 to BIL_5.8
dtypes: category(8), float64(41), int64(8), object(166)
memory usage: 21.4+ MB


Hierbei ist zu sehen, dass 166 Spalten als Datentyp `Objekt` haben und somit auf gemischte Datentypen hindeutet. Im Appendix ist die ausführlicherere Version (@sec-app_df). In diesem ist ersichtlich, dass manche Spalten einige leere Zellen enthalten, so gibt es einige Spalten mit nur 2187 non-null Werten.
Die ersten 10 Zeilen sowie die letzten 10 Zeilen des Datensatz verdeutlichen ebenso, dass nicht alle Werte nutzbar sind (siehe @tbl-erste10 & @tbl-letzte10) <br><br>
Daher führen wir im nächsten Schritt Daten Korrekturen durch, um saubere Datentypen zu haben.

## Data Corrections


- `interger` in `float` verwandeln
- / und - in 0-Werte verwandeln, da diese im engeren Sinne als 0 zählen
- Zahlen in Klammern als normale Zahlen verwandeln

In [5]:
#| code-fold: true
#| echo: true
# integers in float verwandeln
for column in df_bevoelkerung.select_dtypes(['int64']):
    df_bevoelkerung[column] = df_bevoelkerung[column].astype('float64')

df_bevoelkerung = df_bevoelkerung.replace('/',0)
df_bevoelkerung = df_bevoelkerung.replace('-', 0)

for column in df_bevoelkerung.select_dtypes('object'):
    df_bevoelkerung[column]=df_bevoelkerung[column].astype(str).str.extract('(\d+)').astype('float64')

Anschliessend erfolgt der Check, ob die Anpassung der Zahlen, auf Basis zweier bekannter Gemeinden mit ursprünglich nicht korrekt formatierten Werten, nun korrekt ist:

In [6]:
#| echo: true
#| code-fold: true
df_temp = df_bevoelkerung.loc[(df_bevoelkerung['Name'] == 'Barkenholm') | (df_bevoelkerung['Name'] =='Bergewöhrden'), ['DEM_2.7', 'DEM_2.10']]
df_temp

,DEM_2.7,DEM_2.10
43,71.0,9.0
44,19.0,0.0


Die definierten predictor (siehe @sec-pvar)variables müssen im Folgenden noch berechnet werden:

In [7]:
#| echo: true
#| code-fold: true

df_bevoelkerung = df_bevoelkerung.assign(
    Arbeitslosenquote = (1-(df_bevoelkerung['ERW_1.7'] / df_bevoelkerung['ERW_1.4']))*100,
    Migrationsquote = (1-(df_bevoelkerung['MIG_1.2'] / df_bevoelkerung['MIG_1.1']))*100,
    Christenquote = (
        (df_bevoelkerung['REL_1.2'] + df_bevoelkerung['REL_1.3']) 
        / df_bevoelkerung['REL_1.1'])*100,
    Männerquote = ((df_bevoelkerung['DEM_1.2']  / df_bevoelkerung['DEM_1.1'])*100),
    Akademikerquote = (
        (df_bevoelkerung['BIL_5.5'] + df_bevoelkerung['BIL_5.6'] 
        + df_bevoelkerung['BIL_5.7'] + df_bevoelkerung['BIL_5.8'])  / df_bevoelkerung['BIL_5.1'])*100,
    Beamtenquote = (df_bevoelkerung['ERW_2.3'] / df_bevoelkerung['ERW_2.1'])*100,
    Singlequote = (
        (df_bevoelkerung['DEM_2.4'] + df_bevoelkerung['DEM_2.10'] + df_bevoelkerung['DEM_2.13'] 
        + df_bevoelkerung['DEM_2.19'] + df_bevoelkerung['DEM_2.22'] 
        + df_bevoelkerung['DEM_2.25']) / df_bevoelkerung['DEM_2.1'])*100)

### Data splitting
Dataframe auf relevante Spalten kürzen und auf Gemeinde filtern.

Unter Angabe des Spaltenindex filtern wir den Dataframe, sodass wir nur noch die relevanten Spalten erhalten und kopieren diese Werte in ein neues Dataframe `df_analyse`:

In [8]:
#| code-fold: true
#| echo: true
df_analyse = df_bevoelkerung.iloc[:, [6,7,223,224,225,226,227,228,229]].copy()

Mithilfe der `.dropna` Funktion entfernen wir Zeilen deren relevanten Spalten NaN Werte enthalten.

::: {.callout-note}
### Gründe, weshalb NaN vorkommt können folgende sein:
 - Daten sind nicht erhoben worden
 - Zahlenwert der Erfassung nicht sicher genug
 - Aufgrund von Geheimhaltungsverfahren
:::
<br>

Die erneute Ausführung des Codes `pd.info()` ergibt folgende Übersicht.

In [9]:
#| echo: true
#| code-fold: true
df_analyse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Name               12544 non-null  category
 1   Reg_Hier           12544 non-null  category
 2   Arbeitslosenquote  2187 non-null   float64 
 3   Migrationsquote    2187 non-null   float64 
 4   Christenquote      12544 non-null  float64 
 5   Männerquote        12544 non-null  float64 
 6   Akademikerquote    2187 non-null   float64 
 7   Beamtenquote       2187 non-null   float64 
 8   Singlequote        12544 non-null  float64 
dtypes: category(2), float64(7)
memory usage: 1.0 MB


In [10]:
df_analyse.dropna(inplace=True)

Dataframe auf Hierarchie-Ebene **Gemeinde** filtern. Neues Dataframe heißt nun `df_analyse_gemeinde`.

In [11]:
# df_analyse wird über die Spalte Reg_Hier auf Gemeinde gefiltert, der Index zurückgesetzt und eine Kopie erstellt.
df_analyse_gemeinde = df_analyse[df_analyse['Reg_Hier']=='Gemeinde'].reset_index(drop=True).copy()

In [12]:
#| column: page
df_analyse_gemeinde

,Name,Reg_Hier,Arbeitslosenquote,Migrationsquote,Christenquote,Männerquote,Akademikerquote,Beamtenquote,Singlequote
0,"Flensburg, Stadt",Gemeinde,6.657547,15.957447,56.027377,49.276666,13.355639,8.378114,62.147147
1,"Kiel, Landeshauptstadt",Gemeinde,7.539341,18.900021,48.656386,48.139807,17.758138,7.578323,64.350968
2,"Lübeck, Hansestadt",Gemeinde,7.167394,16.812500,56.723806,47.470103,13.992802,6.540654,59.120801
3,"Neumünster, Stadt",Gemeinde,6.899185,16.924489,56.149594,48.816166,8.385235,6.015860,56.965139
4,"Brunsbüttel, Stadt",Gemeinde,5.365854,13.682565,62.607137,49.579243,6.877828,4.123711,51.277856
...,...,...,...,...,...,...,...,...,...
1569,"Greiz, Stadt",Gemeinde,6.813820,2.112338,25.183037,47.736997,13.745338,3.089598,53.023676
1570,"Zeulenroda-Triebes, Stadt",Gemeinde,5.662651,3.613666,28.863238,48.219960,12.058824,4.342273,51.721678
1571,"Altenburg, Stadt",Gemeinde,9.632751,1.978736,13.745967,47.926078,13.866232,3.530979,53.276621
1572,"Meuselwitz, Stadt",Gemeinde,9.363958,2.098540,11.446552,48.403867,8.841463,3.118908,49.872309


Die erneute Ausführung des Codes `pd.info()` ergibt folgende Übersicht. Unser nun bereinigter Dataframe enthält 1573 Zeilen, welcher die Basis für die Anwendung der Modelle bildet.

In [13]:
#| echo: true
#| code-fold: true
df_analyse_gemeinde.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Name               1574 non-null   category
 1   Reg_Hier           1574 non-null   category
 2   Arbeitslosenquote  1574 non-null   float64 
 3   Migrationsquote    1574 non-null   float64 
 4   Christenquote      1574 non-null   float64 
 5   Männerquote        1574 non-null   float64 
 6   Akademikerquote    1574 non-null   float64 
 7   Beamtenquote       1574 non-null   float64 
 8   Singlequote        1574 non-null   float64 
dtypes: category(2), float64(7)
memory usage: 440.7 KB


### Variable List

Im Folgenden werden die Prädikatoren definiert sowie die Outcome-Variable. Zudem werden die Daten für die Prädikatoren sowie die Outcome-Variable definiert. 
<br><br>
Defintion der Prädiktor-Variablen:

In [14]:
#| output: false
#| echo: true
predictor = df_analyse.iloc[:,3:9].columns.values.tolist()

In [15]:
# define outcome variable as y_label
y_label = 'Arbeitslosenquote'

# select features
features = df_analyse.iloc[:,3:9].columns

# create feature data
X = df_analyse_gemeinde[features]

# create response
y = df_analyse_gemeinde[y_label]

### Data Splitting

Für das spätere Modell möchten wir Trainings- und Testdaten. Um die deskriptive und explorative Analyse bereits auf den Trainingsdaten durchzuführen, splitten wir im nächsten Schritt die Daten. Dies machen wir mit der `train_test_split`-Funktion von scikit-learn.

In [16]:
#| echo: true
#| code-fold: true
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [17]:
source = df_analyse_gemeinde

## Descriptive Analytics

Die statistischen Werte, die mithilfer der Funktion `.describe` ausgegeben werden, geben ein erstes Gefühl für die bereingten Daten, welche für die Erstellung der Modelle verwendet werden. 

::: {.callout-note}
### Die Funktion `.describe` enthält:
- Lagemasse (Mittelwert, Median)
- Streuungsmasse (Standardabweichung, Quartile)
:::
<br>

In [18]:
#| column: page
source.describe().applymap('{:,.2f}'.format)

,Arbeitslosenquote,Migrationsquote,Christenquote,Männerquote,Akademikerquote,Beamtenquote,Singlequote
count,"1,574.00","1,574.00","1,574.00","1,574.00","1,574.00","1,574.00","1,574.00"
mean,4.22,17.76,62.14,48.70,13.77,5.07,52.21
std,2.10,9.63,20.94,0.84,5.85,1.74,3.01
min,0.78,0.85,5.93,45.10,2.09,1.24,44.40
25%,2.80,10.66,58.62,48.19,9.78,3.89,50.25
50%,3.62,17.84,68.01,48.70,12.34,4.89,51.69
75%,5.02,24.11,75.94,49.18,16.24,6.02,53.66
max,16.87,53.98,93.91,54.99,48.00,18.87,66.46


## Explorative Analytics

Um die Verteilung der zugrundeliegenden Daten grafisch darstellen zu können, erstellen für für jede Variable ein Histogramm. 
Das Histogramm für die "Christenquote" weist eine linksschiefe, multimodale Verteilung auf. Die "Männerquote" weist eine annähernd symetrische, unimodale Verteilung auf. Alle weiteren Variablen sind rechtsschief, unimodal verteilt.  

In [19]:
#| echo: true
#| code-fold: true
#| column: page-right
alt.Chart(source).mark_bar().encode(
    alt.X(alt.repeat("repeat"), type="quantitative", bin=True),
    y='count()',
).properties(
    width=200,
    height=150 
).repeat(
    repeat=['Arbeitslosenquote', 'Migrationsquote', 'Christenquote', 'Männerquote','Akademikerquote','Beamtenquote','Singlequote'],
    columns = 3
)

alt.RepeatChart(...)

Zur Visualisierung der Beziehungen zwischen Response- und Predictor Variables haben wir uns für die Anwendung von Streudiagrammen bzw. Scatter Plots entschieden. Jeden Prädikator haben wir mit der Arbeitslosenquote gegegnübergestellt, um erste Erkenntnisse gewinnen zu können (siehe @sec-EDArel). Jedoch ist es anhand der Scatter Plots zunächst schwierig zu erkennen, welcher Prädikator die höchste Korrelation mit der Arbeitslosigkeit aufweist, da die berechneten Variablen unterschiedliche Werte bzw. Verteilungen auf der X-Achse annehmen und kein direkter Vergleich stattfinden kann.  <br><br> 


In [20]:
#| echo: true
#| code-fold: true
#| column: page-right
alt.Chart(source, width=200, height=150).mark_circle(size=60).encode(
    alt.X(
        alt.repeat("repeat"), 
        type="quantitative",
        scale=alt.Scale(zero=False)),
    alt.Y('Arbeitslosenquote'),
    tooltip = ['Name',alt.Tooltip(alt.repeat("repeat"), type="quantitative"), alt.Y('Arbeitslosenquote')]
).repeat(
    repeat=predictor,
    columns=3
).interactive()

alt.RepeatChart(...)

Zu erkennen ist, dass lediglich die Variable "Christenquote" eine moderate (über 0,5 bzw. unter -0,5) Korrelation mit der Arbeitslosenquote aufweist. Zwei weitere Quoten, die noch eine leichte bis moderarte Korrelation haben sind die "Singlequote" sowie die "Migrationsquote".<br><br>  

### Pearson Correlation Koeffizienten

Unter Anwendung der Funktion .corr werden die Korrelationen berechnet und in Tabellenform ausgegeben.

In [21]:
#| echo: true
#| code-fold: true
corr_data = source
corr = corr_data.corr(method='pearson').round(5)
corr[y_label].sort_values(ascending=False)
corr.style.background_gradient(cmap='Blues')


,Arbeitslosenquote,Migrationsquote,Christenquote,Männerquote,Akademikerquote,Beamtenquote,Singlequote
Arbeitslosenquote,1.000000,-0.273260,-0.653340,-0.231350,-0.121500,-0.257300,0.426140
Migrationsquote,-0.273260,1.000000,0.417650,-0.050460,0.093190,-0.006110,0.093330
Christenquote,-0.653340,0.417650,1.000000,0.142780,-0.248540,0.275360,-0.285430
Männerquote,-0.231350,-0.050460,0.142780,1.000000,-0.284900,-0.065030,-0.287660
Akademikerquote,-0.121500,0.093190,-0.248540,-0.284900,1.000000,0.301710,0.246270
Beamtenquote,-0.257300,-0.006110,0.275360,-0.065030,0.301710,1.000000,-0.014930
Singlequote,0.426140,0.093330,-0.285430,-0.287660,0.246270,-0.014930,1.000000


Die Pearson Korrelationskoeffizienten bestätigen die oben genannten Vermutungen.

::: {.callout-note}
### Korrelationen
- Moderate Korrelation mit der Arbeitslosenquote: *Christenquote* 
- Leichte bis moderate Korrelation: *Singlequote*
- Ausserst geringe Korrelation: *Akademikerquote*
:::
<br>

# Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

Wie in der Introduction beschrieben haben wir uns aufgrund der Literatur-Recherche wie auch nach Betrachtung des Data Sets für 6 Quoten entschieden, die die Arbeitslosenquote beeinflussen könnten. 

Wie in der explorativen Analyse zu erkannt worden ist, zeigen die Christenquote, Singlequote und Migrationsquote die relativ stärksten Korrelationen. Daher wurde bereits an dieser Stelle beschlossen, keine weiteren Quoten in die Auswahl für die lineare Regression aufzunehmen. Da in den Scatterplot linere Zusammenhänge erkennbar sind, wollen wir versuchen, diese mit lineran Modellen zu erklären.

Für die multiple Regression wie auch die Lasso-Regression haben wir zunächst alle Variablen als Prädikatoren ausgewählt.

Zu Beginn der jeweiligen Modellierungsprozesse haben wir jeweils das Regressionsmodell ausgewählt. Der nächste Schritt war es die Modelle zu validieren und an die Daten anzupassen. 
Die Validierung haben wir mit der Cross-Validation umgesetzt und dabei den Mean Squared error pro Fold angeschaut.
Bei der Lasso-Regression war die Cross-Validation implementiert.

Daraufhin haben wir mit der Funktion `reg.fit` die Modelle an die Traingsdaten angepasst und somit trainiert und den Intercept mit der y-Achse sowie den/die Koeffizienten berechnet.

Der abschließende Schritt war die Evaluierung mit den Test-Daten. Wir haben die Modell dann anhand verschiedener Gütemaße (R^2, MSE, RMSE, MAE) bewertet.

Durch die Peer-Review haben wir das Feedback bekommen, dass wir für die Multiple Regression die Stepwise Selection nutzen könnten sowie für die Bewertung der Güte der Lasso-Regression den AIC. 

Diese grob beschriebenen Schritte wollen wir im Folgenden detaillierter erklären.

## Lineare Regression

### Modell-Auswahl
Zunächst werden für die einzelnen Quoten drei Modelle mit jeweils der linearen Regression definiert. Hier greifen wir auch auf scikit-learn zurück und nehmen `LinearRegression` als Modell.

In [22]:
#| echo: true
reg_chr = LinearRegression() #für Christenquote
reg_mig = LinearRegression() #für Migrationsquote
reg_sin = LinearRegression() #für Singlequote

### Training & Validation
Im nächsten Schritt werden die Modelle trainiert und validiert. Dies wird mit der Cross-Validation durchgeführt. Hierbei berechnen wir für jedes Modell den Mean-Squared-Error für je 5 Folds. Hierfür nehmen wir die Funktion `cross_val_score` und visualisieren das zum einen in einer Tabelle und zum anderen als Liniendiagramm. Hierbei sehen wir, dass die Mean-Squared-Erros je Fold voneinander abweichen, grundsätzlich ist der MSE für die Christenqutote aber immer am geringsten, weshalb wir uns im Rahmen der linearen Regression auf die Christenquote konzentrieren.

In [23]:
# cross-validation with 5 folds
scores_mig = cross_val_score(reg_mig, X_train[['Migrationsquote']], y_train, cv=5, scoring='neg_mean_squared_error') *-1
scores_chr = cross_val_score(reg_chr, X_train[['Christenquote']], y_train, cv=5, scoring='neg_mean_squared_error') *-1
scores_sin = cross_val_score(reg_sin, X_train[['Singlequote']], y_train, cv=5, scoring='neg_mean_squared_error') *-1

In [24]:
# store cross-validation scores: Migrationsquote, Christenquote und Singlequote
df_scores = pd.DataFrame({"lr_mig": scores_mig, "lr_chr": scores_chr,"lr_sin": scores_sin})

# reset index to match the number of folds
df_scores.index += 1

In [25]:
#| column: margin
# print dataframe
df_scores.style.background_gradient(cmap='Blues', axis = 'index')

,lr_mig,lr_chr,lr_sin
1,4.195897,2.222005,3.502820
2,4.199023,2.844424,3.770871
3,3.451805,2.189314,2.852882
4,4.414557,2.539975,4.048354
5,3.951385,2.765169,3.716738


Code Beispiel 42 - Vergleiche Cross Validation Erklärung

In [26]:
alt.Chart(df_scores.reset_index()).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X("index", bin=False, title="Fold", axis=alt.Axis(tickCount=5)),
    y=alt.Y(
        alt.repeat("layer"), aggregate="mean", title="Mean squared error (MSE)"
        ),
        color = alt.datum(alt.repeat("layer"))
).repeat(
    layer = ['lr_mig','lr_chr', 'lr_sin']
)

alt.RepeatChart(...)

In [27]:
df_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
lr_mig,5.0,4.042534,0.368673,3.451805,3.951385,4.195897,4.199023,4.414557
lr_chr,5.0,2.512177,0.301499,2.189314,2.222005,2.539975,2.765169,2.844424
lr_sin,5.0,3.578333,0.449758,2.852882,3.502820,3.716738,3.770871,4.048354


### Fit Model
Im nächsten Schritt möchten wir das Modell an die Trainingsdaten anpassen und den y-Achsenabschnitt sowie die Steigung berechnen. Hierfür nutzen wir die `.fit` Funktion.

In [28]:
#| output: false
# Fit the model to the complete training data
reg_chr.fit(X_train[['Christenquote']], y_train)

LinearRegression()

Christenquote:

In [29]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg_chr.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": 'slope',
    "Coefficient": reg_chr.coef_}
)

# combine estimates of intercept and slope
table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,8.194
1,slope,-0.065


### Evaluation on test set
Im nächsten Schritt evaulieren wir unser Modell mit den Testdaten. Dazu prognostizieren wir y-Werte, das bedeutet Arbeitslosigkeitsquoten auf Basis der Testdaten, welche diverse Christenquoten darstellen. Dies setzen wir mit der Funktion `.predict` um. 

In [30]:
y_pred_chr = reg_chr.predict(X_test[['Christenquote']])

Für die prognostizierten Werte berechnen wir Gütemaße, wie den R-squared (R2), den Mean-Squared-Error (MSE), den Rooted-Mean-Squared-Error (RMSE) sowie den Mean-Absolute-Error (MAE).

In [31]:
r2_chr = r2_score(y_test, y_pred_chr).round(3)
mse_chr = mean_squared_error(y_test, y_pred_chr).round(3)
rmse_chr = mean_squared_error(y_test, y_pred_chr, squared=False).round(3)
mae_chr = mean_absolute_error(y_test, y_pred_chr).round(3)

results = pd.DataFrame(
    {'Christenquote': [
        r2_chr,
        mse_chr,
        rmse_chr,
        mae_chr
    ]},
    index = [
        'R2',
        'MSE',
        'RMSE',
        'MAE'
    ]
)
results

,Christenquote
R2,0.418
MSE,2.613
RMSE,1.616
MAE,1.194


## Multiple Regression

### Modell-Auswahl
Für die Multiple Regression nehmen wir auch von scikit-learn die `LinearRegression` als Modell. Der Unterschied zum vorherigen Abschnitt ist allerdings, dass wir hierbei mehr als eine Predictor-Variable nehmen.

In [32]:
#| echo: true
#| output: false
reg_multi=LinearRegression()

### Training & Validation
Zuerst nehmen wir alle sechs predictor-Variablen, um das Modell der multiplen Regression zu trainieren und validieren. Hierfür nutzen wir den gleichen Cross-Validation Ansatz wie bei der linearen Regression.

In [33]:
#| echo: true
#| code-fold: true
#| column: margin
# cross-validation with 5 folds total
scores_multi = cross_val_score(reg_multi, X_train, y_train, cv=5, scoring='neg_mean_squared_error') *-1
df_scores = df_scores.assign(lr_multi = scores_multi)
df_scores.style.background_gradient(cmap = 'Blues', axis='index')

,lr_mig,lr_chr,lr_sin,lr_multi
1,4.195897,2.222005,3.502820,1.505448
2,4.199023,2.844424,3.770871,1.640132
3,3.451805,2.189314,2.852882,1.348555
4,4.414557,2.539975,4.048354,1.617988
5,3.951385,2.765169,3.716738,1.884595


Im nächsten Schritt möchten wir die Wrapper-Methoden nutzen, um zu prüfen, ob die Multiple Regression mit weniger Features besser performt und somit leichter verständlich wird. Hierfür nutzen wir von scikit-learn den `SequentialFeatureSelector`. Nachdem wir die Anzahl der ausgewählten Features selektiert haben und den MSE für je 5 Folds angeschaut haben, kamen wir zu der Erkenntnis, dass die Multiple Regression mit der Anzahl von 5 Features am Besten performt. Diese Variationen haben wir mit der Forward- und Backward-Selection durchgeführt.

In [34]:
sfs_backward = SequentialFeatureSelector(
    reg_multi, n_features_to_select=5, cv=5, direction="backward"
).fit(X_train, y_train)

In [35]:
sfs_forward = SequentialFeatureSelector(
    reg_multi, n_features_to_select=5, cv=5, direction="forward"
).fit(X_train, y_train)

In [36]:
features_selection = sfs_backward.get_support()

Das Ergebnis der Backward- und Forward-Selection sind jeweils die gleichen fünf Features. Das ist eher untypisch. Obwohl der `SequentialFeatureSelector` bereits eine Cross-Validation mit 5 Folds durchführt, machen wir das für die drei selektieren Variablen auch noch, um die beiden multiplen Regression zu vergleichen.

In [37]:
#| echo: true
#| code-fold: true
#| column: margin
# cross-validation with 5 folds total
scores_multi5 = cross_val_score(reg_multi, X_train.iloc[:,features_selection], y_train, cv=5, scoring='neg_mean_squared_error') *-1
df_scores = df_scores.assign(lr_multi5 = scores_multi5)
df_scores.style.background_gradient(cmap = 'Blues', axis='index')

,lr_mig,lr_chr,lr_sin,lr_multi,lr_multi5
1,4.195897,2.222005,3.502820,1.505448,1.497303
2,4.199023,2.844424,3.770871,1.640132,1.635291
3,3.451805,2.189314,2.852882,1.348555,1.346716
4,4.414557,2.539975,4.048354,1.617988,1.615941
5,3.951385,2.765169,3.716738,1.884595,1.884977


:::{.callout-note}
## Erkenntnis
Hierbei sehen wir, dass die Mean-Squared-Erros für die Multiple Regression mit 5 Features niedriger sind, weshalb wir im weiteren Verlauf diese 5 ausgewählten für die Multiple Regression berücksichtigen werden. 
:::

In [38]:
df_test = pd.DataFrame({'Features': features[features_selection]})
df_test.index +=1
df_test

,Features
1,Christenquote
2,Männerquote
3,Akademikerquote
4,Beamtenquote
5,Singlequote


### Fit Model
In diesem Schritt werden wir wieder das Modell an die Trainingsdaten anpassen und den y-Achsenabschnitt sowie die Steigung berechnen.

In [39]:
# Fit the model to the complete training data
reg_multi.fit(X_train.iloc[:,features_selection], y_train)

LinearRegression()

In [40]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg_multi.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": features[features_selection],
    "Coefficient": reg_multi.coef_}
)

# combine estimates of intercept and slopes
table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,20.179
1,Christenquote,-0.065
2,Männerquote,-0.427
3,Akademikerquote,-0.149
4,Beamtenquote,0.044
5,Singlequote,0.204


### Evaluation on test set
Im nächsten Schritt evaulieren wir unser Modell mit den Testdaten. Dazu prognostizieren wir y-Werte, das bedeutet Arbeitslosigkeitsquoten auf Basis der Testdaten, welche diverse Christenquoten darstellen. Dies setzen wir erneut mit der Funktion `.predict` um. 

In [41]:
#obtain predictions
y_pred_multi = reg_multi.predict(X_test.iloc[:,features_selection])

Für die prognostizierten Werte berechnen wir Gütemaße, wie den R-squared (R2), den Mean-Squared-Error (MSE), den Rooted-Mean-Squared-Error (RMSE) sowie den Mean-Absolute-Error (MAE).

In [42]:
r2_multi = r2_score(y_test, y_pred_multi).round(3)
mse_multi = mean_squared_error(y_test, y_pred_multi).round(3)
rmse_multi = mean_squared_error(y_test, y_pred_multi, squared=False).round(3)
mae_multi = mean_absolute_error(y_test, y_pred_multi).round(3)
results = results.assign(MultipleRegression = [r2_multi,mse_multi,rmse_multi,mae_multi])

In [43]:
results

,Christenquote,MultipleRegression
R2,0.418,0.625
MSE,2.613,1.683
RMSE,1.616,1.297
MAE,1.194,0.974


## Lasso Regression

### Modell-Auswahl
Bei der Lasso Regression ist es zuerst notwendig, die Variablen zu standardisieren, da Lasso am Besten performt, wenn die Features um den Wert 0 zentriert sind. Für diese Standardisierung definieren wir zuerst mit der `StandardScaler().fit`-Funktion unseren Skalierer. Diese scikit-learn Funktion entfernt den Mittelwert und skaliert jedes Feature auf eine Einheitsvarianz. Das wird für jedes Feature getrennt durchgeführt. 
Im nächsten Schritt transformieren wir unsere Trainings- und Testdaten mit diesem Skalierer.
Für die Berechnung des alpha-Wertes für Lasso Regression nehmen wir von scikit-learn die `LassoCV`-Funktion, welche bereits eine Cross-Validation integriert.

In [44]:
#| code-fold: true
#| echo: true
X_train_lasso = X_train.copy()
X_test_lasso = X_test.copy()
scaler = StandardScaler().fit(X_train[features]) 

X_train_lasso[features] = scaler.transform(X_train_lasso[features])
X_test_lasso[features] = scaler.transform(X_test_lasso[features])

In [45]:
#| echo: true
#| output: false
# select the lasso model with built in crossvalidation
reg_lasso = LassoCV(cv=5, random_state=0)

### Training & best alpha
Die `LassoCV`-Funktion hat wie bereits erwähnt, die Cross Validation integriert. Somit ermittelt das Model für uns den bestmöglichen Alpha-Wert um die Regression durchzuführen. Nachdem wir unser Modell trainiert haben, können wir uns mit `.alpha_` den bestmöglichen Alpha-Wert ausgeben lassen.

In [46]:
#| output: false
#| echo: true
reg_lasso.fit(X_train_lasso, y_train)

LassoCV(cv=5, random_state=0)

In [47]:
reg_lasso.alpha_

0.00730882363957819

### Fit Model
Anschließend nutzen wir den besten Wert für Alpha um die Lasso Regression damit durchzuführen. Dafür definieren wir für die Regression das alpha.

In [48]:
#| echo: true
#| output: false
# Fit the model to the complete training data
lasso_best = Lasso(alpha=reg_lasso.alpha_)
lasso_best.fit(X_train_lasso, y_train)

Lasso(alpha=0.00730882363957819)

In [49]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[lasso_best.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": features,
    "Coefficient": lasso_best.coef_}
)

# combine estimates of intercept and slopes
table = pd.concat([intercept, slope], ignore_index=True, sort=False)
round(table, 3)

,Name,Coefficient
0,Intercept,4.201
1,Migrationsquote,0.000
2,Christenquote,-1.373
3,Männerquote,-0.353
4,Akademikerquote,-0.856
5,Beamtenquote,0.060
6,Singlequote,0.609


### Evaluation on test set
Wie in den vorherigen beiden Modellen evaluieren wir unser Modell mit den Test-Daten und prognostizieren Werte.

Für die prognostizierten Werte berechnen wir Gütemaße, wie den R-squared (R2), den Mean-Squared-Error (MSE), den Rooted-Mean-Squared-Error (RMSE) sowie den Mean-Absolute-Error (MAE).

In [50]:
# obtain predictions
y_pred_lasso = lasso_best.predict(X_test_lasso)

In [51]:
r2_lasso = r2_score(y_test, y_pred_lasso).round(3)
mse_lasso = mean_squared_error(y_test, y_pred_lasso).round(3)
rmse_lasso = mean_squared_error(y_test, y_pred_lasso, squared=False).round(3)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso).round(3)
results = results.assign(Lasso = [r2_lasso,mse_lasso,rmse_lasso,mae_lasso])

In [52]:
results

,Christenquote,MultipleRegression,Lasso
R2,0.418,0.625,0.625
MSE,2.613,1.683,1.686
RMSE,1.616,1.297,1.298
MAE,1.194,0.974,0.973


# Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.

Mithilfe der durchgeführten Analyse konnte festgestellt werden, dass die stärkste positive Korrelation (nach Pearson) zwischen der Arbeitslosenquote einer Gemeinde und dem Prädikator "Singlequote" besteht (r = +0.426). Die stärkste negative Korrelation besteht zwischen der Arbeitslosenquote einer Gemeinde und dem Prädikator "Christenquote" (r = -0.653). Die schwächste Korrelation, auf Basis unserer Daten, weist die "Akademikerquote" auf (r=-0.121).

Das Ergebnis der drei durchgeführten Modelle lautet wie folgt: 


In [53]:
#| echo: true
#| output: false
results

,Christenquote,MultipleRegression,Lasso
R2,0.418,0.625,0.625
MSE,2.613,1.683,1.686
RMSE,1.616,1.297,1.298
MAE,1.194,0.974,0.973


Die Ergebnisse lassen darauf schliessen, dass entweder die Prädikatoren nicht aussagekräftig genug sind, um die Arbeitslosigkeit erklären zu können oder das ein anderes statistisches Modell gewählt werden muss.

Folgende Herausforderungen gab es bei der Durchführung des Projekts: 

:::{.callout-note}
## Herausforderungen:
1. Verknüpfung zweier Dataframes
2. Grosser Dataframe, obwohl nur wenige Daten tatsächlich relevant waren.
3. Ein Grossteil der Zellen relevanter Spalten war leer, da durch die Zensus Umfrage die Werte entweder nicht ermittelt werden konnten oder zu ungenau waren. 
4. Geringe Korrelation zwischen Arbeitslosenquote und den Variablen.
:::

# Discussion + Conclusion


Aufgrund der durchgeführten Data Correction wurden die tatsächlich genutzten Zeilen bzw. Gemeinden stark reduziert. Von insgesamt 11.339 Gemeinden wurden schlussendlich nur 1.573 Gemeinden berücksichtigt. Daher wäre es spannend zu wissen, ob die berücksichtigten Gemeinden repräsentativ für ganz Deutschland sind. Hierzu analysieren wir den original Dataframe mit dem bereinigten Dataframe. 

## Bereinigter DF

In [73]:
#Filter auf Anzahl Einwohner, Anzahl Erwerbspersonen und Anzahl Erwerbstätige pro Gemeinde

#| echo: true 
#| code-fold: true
#Summen für den bereingten DF
df_analyse = df_bevoelkerung[['Reg_Hier','AEWZ','ERW_1.4','ERW_1.7']].copy()
df_analyse.rename(columns = {'Reg_Hier' : 'Region_Hierarchie', 'AEWZ' : 'Anzahl Einwohner', 'ERW_1.4' : 'Anzahl Erwerbspersonen', 'ERW_1.7' : 'Anzahl Erwerbstätige'}, inplace = True)
df_analyse

,Region_Hierarchie,Anzahl Einwohner,Anzahl Erwerbspersonen,Anzahl Erwerbstätige
0,Bund,80219695.0,43052760.0,41049730.0
1,Land,2800119.0,1481080.0,1413270.0
2,Gemeinde,82258.0,43860.0,40940.0
3,Stadtkreis/kreisfreie Stadt/Landkreis,82258.0,43860.0,40940.0
4,Gemeinde,235782.0,127730.0,118100.0
...,...,...,...,...
12539,Gemeinde,307.0,NaN,NaN
12540,Gemeindeverband,5463.0,NaN,NaN
12541,Gemeinde,3724.0,NaN,NaN
12542,Gemeinde,138.0,NaN,NaN


In [81]:
#Filter auf Ebene Gemeinde
#| echo: true 
#| code-fold: true
df_analyse_gemeinde = df_analyse[df_analyse['Region_Hierarchie']=='Gemeinde'].reset_index(drop=True).copy()
df_analyse_gemeinde

,Region_Hierarchie,Anzahl Einwohner,Anzahl Erwerbspersonen,Anzahl Erwerbstätige
0,Gemeinde,82258.0,43860.0,40940.0
1,Gemeinde,235782.0,127730.0,118100.0
2,Gemeinde,210305.0,107710.0,99990.0
3,Gemeinde,77249.0,39280.0,36570.0
4,Gemeinde,12834.0,6150.0,5820.0
...,...,...,...,...
11334,Gemeinde,329.0,NaN,NaN
11335,Gemeinde,307.0,NaN,NaN
11336,Gemeinde,3724.0,NaN,NaN
11337,Gemeinde,138.0,NaN,NaN


In [82]:
#| echo: true 
#| code-fold: true
df_analyse_gemeinde.dropna(inplace=True)
df_analyse_gemeinde

,Region_Hierarchie,Anzahl Einwohner,Anzahl Erwerbspersonen,Anzahl Erwerbstätige
0,Gemeinde,82258.0,43860.0,40940.0
1,Gemeinde,235782.0,127730.0,118100.0
2,Gemeinde,210305.0,107710.0,99990.0
3,Gemeinde,77249.0,39280.0,36570.0
4,Gemeinde,12834.0,6150.0,5820.0
...,...,...,...,...
11289,Gemeinde,21034.0,10420.0,9710.0
11295,Gemeinde,15421.0,8300.0,7830.0
11299,Gemeinde,34090.0,16610.0,15010.0
11301,Gemeinde,10964.0,5660.0,5130.0


In [83]:
#| echo: true 
#| code-fold: true
Summe_bereinigt = df_analyse_gemeinde[['Anzahl Einwohner','Anzahl Erwerbspersonen','Anzahl Erwerbstätige']].sum()
Summe_bereinigt

Anzahl Einwohner          58797595.0
Anzahl Erwerbspersonen    31235700.0
Anzahl Erwerbstätige      29629470.0
dtype: float64

In [84]:
#| echo: true 
#| code-fold: true
# Boxplot 
box = alt.Chart(df_analyse_gemeinde).mark_boxplot().encode(
    x='Anzahl Einwohner',
)
box

alt.Chart(...)

## Original DF

In [74]:
#| echo: true 
#| code-fold: true
#Filter auf Anzahl Einwohner, Anzahl Erwerbspersonen und Anzahl Erwerbstätige pro Gemeinde

df_analyse = df_bevoelkerung[['Reg_Hier','AEWZ','ERW_1.4','ERW_1.7']].copy()
df_analyse.rename(columns = {'Reg_Hier' : 'Region_Hierarchie', 'AEWZ' : 'Anzahl Einwohner', 'ERW_1.4' : 'Anzahl Erwerbspersonen', 'ERW_1.7' : 'Anzahl Erwerbstätige'}, inplace = True)
df_analyse

,Region_Hierarchie,Anzahl Einwohner,Anzahl Erwerbspersonen,Anzahl Erwerbstätige
0,Bund,80219695.0,43052760.0,41049730.0
1,Land,2800119.0,1481080.0,1413270.0
2,Gemeinde,82258.0,43860.0,40940.0
3,Stadtkreis/kreisfreie Stadt/Landkreis,82258.0,43860.0,40940.0
4,Gemeinde,235782.0,127730.0,118100.0
...,...,...,...,...
12539,Gemeinde,307.0,NaN,NaN
12540,Gemeindeverband,5463.0,NaN,NaN
12541,Gemeinde,3724.0,NaN,NaN
12542,Gemeinde,138.0,NaN,NaN


In [76]:
#| echo: true 
#| code-fold: true
#Filter auf Ebene Gemeinde
df_analyse_gemeinde_orig = df_analyse[df_analyse['Region_Hierarchie']=='Gemeinde'].reset_index(drop=True).copy()
df_analyse_gemeinde_orig

,Region_Hierarchie,Anzahl Einwohner,Anzahl Erwerbspersonen,Anzahl Erwerbstätige
0,Gemeinde,82258.0,43860.0,40940.0
1,Gemeinde,235782.0,127730.0,118100.0
2,Gemeinde,210305.0,107710.0,99990.0
3,Gemeinde,77249.0,39280.0,36570.0
4,Gemeinde,12834.0,6150.0,5820.0
...,...,...,...,...
11334,Gemeinde,329.0,NaN,NaN
11335,Gemeinde,307.0,NaN,NaN
11336,Gemeinde,3724.0,NaN,NaN
11337,Gemeinde,138.0,NaN,NaN


In [78]:
#| echo: true 
#| code-fold: true
# Boxplot 
box = alt.Chart(df_analyse_gemeinde_orig).mark_boxplot().encode(
    x='Anzahl Einwohner',
)
box

alt.Chart(...)

In [79]:
#| echo: true 
#| code-fold: true
Summe = df_analyse_gemeinde_orig[['Anzahl Einwohner','Anzahl Erwerbspersonen','Anzahl Erwerbstätige']].sum()
Summe

Anzahl Einwohner          80209997.0
Anzahl Erwerbspersonen    31235700.0
Anzahl Erwerbstätige      29629470.0
dtype: float64

Beim Vergleich der Anzahl Einwohner zwischen originalem DF und bereinigtem DF, fällt auf, dass in allen Gemeinden 80.029.997 Einwohner leben. Der bereinigte DF enthält jedoch nur 58.797.595 Einwohner. Die entsprich ca. 73.47 % der gesamten Einwohnerzahl Deutschlands. Die Sumemr aller Gemeinden beträgt 11.339 Stück. Bereinigt wurden jedoch nur 1.573 Gemeinden berücksichtigt, was einem Anteil von ca. 13.87 % entspricht. 

:::{.callout-note}
## Erkenntnis
Dies lässt darauf schliessen, dass in nur 13.87% der Gemeinden Deutschlands 73.47% der Bevölkerung lebt. Somit wurden für die Untersuchung der Fragestellung überwiegend bevölkerungsreiche Gemeinden berücksichtigt, da kleine Gemeinden tendeziell weniger Angaben machten und somit herausgefiltert wurden.  
:::

# Appendix

## Appendix df_bevoelkerung {#sec-app_df}

In [63]:
df_bevoelkerung.info(verbose = True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Data columns (total 230 columns):
 #    Column             Non-Null Count  Dtype   
---   ------             --------------  -----   
 0    AGS_12             12544 non-null  category
 1    RS_Land            12544 non-null  category
 2    RS_RB_NUTS2        12527 non-null  category
 3    RS_Kreis           12501 non-null  category
 4    RS_VB              12089 non-null  category
 5    RS_Gem             11339 non-null  category
 6    Name               12544 non-null  category
 7    Reg_Hier           12544 non-null  category
 8    AEWZ               12544 non-null  float64 
 9    DEM_1.1            12544 non-null  float64 
 10   DEM_1.2            12544 non-null  float64 
 11   DEM_1.3            12544 non-null  float64 
 12   DEM_2.1            12544 non-null  float64 
 13   DEM_2.2            12544 non-null  float64 
 14   DEM_2.3            12544 non-null  float64 
 15   DEM_2.4            12544 non-null 

In [64]:
#| label: tbl-erste10
#| tbl-cap: "Erste zehn Zeilen Datensatz"
df_bevoelkerung.head(10)

,AGS_12,RS_Land,RS_RB_NUTS2,RS_Kreis,RS_VB,RS_Gem,Name,Reg_Hier,AEWZ,DEM_1.1,...,BIL_5.6,BIL_5.7,BIL_5.8,Arbeitslosenquote,Migrationsquote,Christenquote,Männerquote,Akademikerquote,Beamtenquote,Singlequote
0,0,0,NaN,NaN,NaN,NaN,Deutschland,Bund,80219695.0,80219695.0,...,3985640.0,5471080.0,908970.0,4.652501,19.205282,59.132542,48.798417,16.541543,5.080131,54.203097
1,1,1,NaN,NaN,NaN,NaN,Schleswig-Holstein,Land,2800119.0,2800119.0,...,126590.0,157620.0,25380.0,4.578416,12.024768,57.512020,48.586649,14.198910,6.676007,54.187161
2,10010000000,1,0,1,0,0,"Flensburg, Stadt",Gemeinde,82258.0,82258.0,...,3370.0,6210.0,0.0,6.657547,15.957447,56.027377,49.276666,13.355639,8.378114,62.147147
3,1001,1,0,1,NaN,NaN,"Flensburg, Stadt",Stadtkreis/kreisfreie Stadt/Landkreis,82258.0,82258.0,...,3370.0,6210.0,0.0,6.657547,15.957447,56.027377,49.276666,13.355639,8.378114,62.147147
4,10020000000,1,0,2,0,0,"Kiel, Landeshauptstadt",Gemeinde,235782.0,235782.0,...,10690.0,19320.0,4100.0,7.539341,18.900021,48.656386,48.139807,17.758138,7.578323,64.350968
5,1002,1,0,2,NaN,NaN,"Kiel, Landeshauptstadt",Stadtkreis/kreisfreie Stadt/Landkreis,235782.0,235782.0,...,10690.0,19320.0,4100.0,7.539341,18.900021,48.656386,48.139807,17.758138,7.578323,64.350968
6,10030000000,1,0,3,0,0,"Lübeck, Hansestadt",Gemeinde,210305.0,210305.0,...,8490.0,13120.0,1800.0,7.167394,16.812500,56.723806,47.470103,13.992802,6.540654,59.120801
7,1003,1,0,3,NaN,NaN,"Lübeck, Hansestadt",Stadtkreis/kreisfreie Stadt/Landkreis,210305.0,210305.0,...,8490.0,13120.0,1800.0,7.167394,16.812500,56.723806,47.470103,13.992802,6.540654,59.120801
8,10040000000,1,0,4,0,0,"Neumünster, Stadt",Gemeinde,77249.0,77249.0,...,2630.0,2230.0,0.0,6.899185,16.924489,56.149594,48.816166,8.385235,6.015860,56.965139
9,1004,1,0,4,NaN,NaN,"Neumünster, Stadt",Stadtkreis/kreisfreie Stadt/Landkreis,77249.0,77249.0,...,2630.0,2230.0,0.0,6.899185,16.924489,56.149594,48.816166,8.385235,6.015860,56.965139


In [65]:
#| label: tbl-letzte10
#| tbl-cap: "Letzte zehn Zeilen Datensatz"
df_bevoelkerung.tail(10)

,AGS_12,RS_Land,RS_RB_NUTS2,RS_Kreis,RS_VB,RS_Gem,Name,Reg_Hier,AEWZ,DEM_1.1,...,BIL_5.6,BIL_5.7,BIL_5.8,Arbeitslosenquote,Migrationsquote,Christenquote,Männerquote,Akademikerquote,Beamtenquote,Singlequote
12534,"1,60775E+11",16,0,77,5009,26,Löbichau,Gemeinde,1041.0,1041.0,...,NaN,NaN,NaN,NaN,NaN,27.377522,49.567723,NaN,NaN,48.703170
12535,"1,60775E+11",16,0,77,5009,37,Nöbdenitz,Gemeinde,938.0,938.0,...,NaN,NaN,NaN,NaN,NaN,23.667377,50.639659,NaN,NaN,44.349680
12536,"1,60775E+11",16,0,77,5009,41,Posterstein,Gemeinde,447.0,446.0,...,NaN,NaN,NaN,NaN,NaN,17.264574,52.466368,NaN,NaN,43.497758
12537,"1,60775E+11",16,0,77,5009,47,Thonhausen,Gemeinde,571.0,570.0,...,NaN,NaN,NaN,NaN,NaN,47.368421,50.526316,NaN,NaN,47.192982
12538,"1,60775E+11",16,0,77,5009,49,Vollmershain,Gemeinde,329.0,328.0,...,NaN,NaN,NaN,NaN,NaN,46.341463,50.609756,NaN,NaN,48.170732
12539,"1,60775E+11",16,0,77,5009,51,Wildenbörten,Gemeinde,307.0,307.0,...,NaN,NaN,NaN,NaN,NaN,39.087948,50.488599,NaN,NaN,48.208469
12540,160775050,16,0,77,5050,NaN,"Gößnitz, Stadt",Gemeindeverband,5463.0,5463.0,...,NaN,NaN,NaN,NaN,NaN,24.492037,49.258649,NaN,NaN,50.210507
12541,"1,60775E+11",16,0,77,5050,12,"Gößnitz, Stadt",Gemeinde,3724.0,3724.0,...,NaN,NaN,NaN,NaN,NaN,22.314715,48.335124,NaN,NaN,52.309345
12542,"1,60775E+11",16,0,77,5050,17,Heyersdorf,Gemeinde,138.0,139.0,...,NaN,NaN,NaN,NaN,NaN,37.410072,56.834532,NaN,NaN,47.482014
12543,"1,60775E+11",16,0,77,5050,39,Ponitz,Gemeinde,1601.0,1600.0,...,NaN,NaN,NaN,NaN,NaN,28.437500,50.750000,NaN,NaN,45.562500


## Appendix Predictor Variables

### Predictor Variables {#sec-pvar}

In [66]:
df_predictor_variables = pd.read_excel('../references/Predictor Variables Definition.xlsx', index_col= 0)

In [67]:
left_align(df_predictor_variables)

,Quote,Berechnung
Variable,,
Migrationshintergrund,Migrationsquote,(Anzahl Personen mit Migrationshintergrund / Anzahl Personen insgesamt)
Religionszugehörigkeit,Christenquote*,(Römisch-katholische Kirche + Evangelische Kirch) / Bevölkerung nach Religion gesamt
Geschlecht,Männerquote,(Anzahl Männer / Einwohner gesamt)
Bildungsniveau,Akademikerquote**,(Fach- oder Berufsakademie + FH-Abschluss + Hochschulabschluss + Promotion) / höchster beruflicher Abschluss insgesamt
Stellung im Beruf,Beamtenquote,(Anzahl Beamter / Erwerbstätige insgesamt)
Familienstand,Singlequote***,(Anzahl Lediger + Verwitwete + Geschiedene + eingetragene Lebenspartnerschaft aufgehoben + Eingetragener Lebenspartner/- in verstorben + ohne Angaben) / Familienstand gesamt


## Appendix EDA

### EDA {#sec-EDArel}